https://github.com/luxonis/depthai-experiments/tree/master/gen2-gaze-estimation

In [ ]:
while True:
    pass

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

##### Adapt my folder structure

###### Sort

In [ ]:
"""
import pandas as pd

bees=pd.read_csv("/content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/bee_data.csv", 
                index_col=False,  
                parse_dates={'datetime':[1,2]},
                dtype={'subspecies':'category', 'health':'category','caste':'category'})

for index, bee in bees.iterrows():
    file = "/content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/bee_imgs/" + bee['file']
    print(file)

    health = bee['health']
    folder = ""
    if health == "healthy":
        folder = "/content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted/healthy/"
    if health == "Varroa, Small Hive Beetles" or "few varrao, hive beetles":
        folder = "/content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted/varroa/"
    if health == "hive being robbed":
        folder = "/content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted/robbed/"
    if health == "ant problems":
        folder = "/content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted/ants/"
    if health == "missing queen":
        folder = "/content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted/noqueen/"

    !cp {file} {folder}
"""

###### Balance and Sort

In [ ]:
"""
!rm -r /content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted/ants/*
!rm -r /content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted/healthy/*
!rm -r /content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted/noqueen/*
!rm -r /content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted/robbed/*
!rm -r /content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted/varroa/*

import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

def balance(bees, field_name):
    #Balance by subspecies to train_bees_bal_ss dataset
    # Number of samples in each category
    ncat_bal = int(len(bees)/bees[field_name].cat.categories.size)
    bees_bal = bees.groupby(field_name, as_index=False).apply(lambda g:  g.sample(ncat_bal, replace=True)).reset_index(drop=True)
    return(bees_bal)
    
def plot_balanced(unbalanced, balanced, field_name):
    # Plot before and after balancing
    f, axs = plt.subplots(1,2, figsize=(8,4))

    ax = unbalanced[field_name].value_counts().plot(kind='bar', ax=axs[0])
    ax.set_title('%s before balancing' % field_name)
    ax.set_ylabel('Count')

    ax = balanced[field_name].value_counts().plot(kind='bar', ax=axs[1])
    ax.set_title('%s after balancing' % field_name)
    ax.set_ylabel('Count')

    plt.tight_layout()
    plt.show()

bees=pd.read_csv("/content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/bee_data.csv", 
                index_col=False,  
                parse_dates={'datetime':[1,2]},
                dtype={'subspecies':'category', 'health':'category','caste':'category'})

# Split/balance and plot the result
bees_balanced = balance(bees, 'health')
plot_balanced(bees, bees_balanced, 'health')

# Will use balanced dataset as main
bees = bees_balanced

print(bees['health'].value_counts())

for (index, bee) in tqdm(bees.iterrows()):
    file = "/content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/bee_imgs/" + bee['file']
    #print(file)

    health = bee['health']
    folder = ""
    if health == "Varroa, Small Hive Beetles" or health == "few varrao, hive beetles":
        folder = "/content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted/varroa/"
    if health == "healthy":
        folder = "/content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted/healthy/"
    if health == "hive being robbed":
        folder = "/content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted/robbed/"
    if health == "ant problems":
        folder = "/content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted/ants/"
    if health == "missing queen":
        folder = "/content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted/noqueen/"

    !cp {file} {folder}
"""

Note that due to varroa being in there twice, the result is still unbalanced.
Varroa is double as likely, so:


*   Healthy - 16,6%
*   Varroa - 33,2%
*   Ants - 16,6%
*   no Queen - 16,6%
*   being robbed - 16,6%


In [ ]:
"""
# BONUS: varroa vs. healthy only
!cp /content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted/varroa/* /content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/varroavshealthy/varroa/

# using ants also as healthy
# this balances the dataset
# and allows to distinguish between ants and varroa
!cp /content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted/healthy/* /content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/varroavshealthy/healthy/
!cp /content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted/ants/* /content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/varroavshealthy/healthy/
"""

##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image classification

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/images/classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/images/classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial shows how to classify images of flowers. It creates an image classifier using a `keras.Sequential` model, and loads data using `preprocessing.image_dataset_from_directory`. You will gain practical experience with the following concepts:

* Efficiently loading a dataset off disk.
* Identifying overfitting and applying techniques to mitigate it, including data augmentation and Dropout.

This tutorial follows a basic machine learning workflow:

1. Examine and understand data
2. Build an input pipeline
3. Build the model
4. Train the model
5. Test the model
6. Improve the model and repeat the process

## Import TensorFlow and other libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

## Download and explore the dataset

This uses a dataset of about 862*6 photos of bees. The dataset contains 5 sub-directories, one per class:

```
sorted/
  ants/
  healthy/
  noqueen/
  robbed/
  varroa/
```

In [ ]:
import pathlib
#data_dir = pathlib.Path("/content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/varroavshealthy")
data_dir = pathlib.Path("/content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted")

# use this, if you want all health possibilities: healthy, varroa, queen, ants, robbed
# check the order!
#data_dir = pathlib.Path("/content/gdrive/MyDrive/TGMB/datasets/honey-bee-annotated-images/sorted")

After downloading, you should now have a copy of the dataset available. There are 2262 total images, but only 1992 are used:

In [ ]:
image_count = len(list(data_dir.glob('*/*.png'))) # note, these are png not jpg
print(image_count)

Here are some healthy bees:

In [ ]:
healthy = list(data_dir.glob('healthy/*'))
PIL.Image.open(str(healthy[0]))

In [ ]:
PIL.Image.open(str(healthy[1]))

And some with varroa:

In [ ]:
varroa = list(data_dir.glob('varroa/*'))
PIL.Image.open(str(varroa[0]))

In [ ]:
PIL.Image.open(str(varroa[1]))

# Load using keras.preprocessing

Let's load these images off disk using the helpful [image_dataset_from_directory](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory) utility. This will take you from a directory of images on disk to a `tf.data.Dataset` in just a couple lines of code. If you like, you can also write your own data loading code from scratch by visiting the [load images](https://www.tensorflow.org/tutorials/load_data/images) tutorial.

## Create a dataset

Define some parameters for the loader:

Note the extremely small image size.
This is ok, because the base image for the bee detection is 300x300 (or 320?).
A bee will never fill the whole base image, so the cut out bee will be even smaller.

In [ ]:
batch_size = 32
img_height = 50 #180
img_width = 50 #180

It's good practice to use a validation split when developing your model. Let's use 80% of the images for training, and 20% for validation.

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

You can find the class names in the `class_names` attribute on these datasets. These correspond to the directory names in alphabetical order.

In [ ]:
class_names = train_ds.class_names
print(class_names)

## Configure the dataset for performance

Let's make sure to use buffered prefetching so you can yield data from disk without having I/O become blocking. These are two important methods you should use when loading data.

`Dataset.cache()` keeps the images in memory after they're loaded off disk during the first epoch. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache.

`Dataset.prefetch()` overlaps data preprocessing and model execution while training. 

Interested readers can learn more about both methods, as well as how to cache data to disk in the [data performance guide](https://www.tensorflow.org/guide/data_performance#prefetching).

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Standardize the data

This is just informational.
You may skip it.

The RGB channel values are in the `[0, 255]` range. This is not ideal for a neural network; in general you should seek to make your input values small. Here, you will standardize values to be in the `[0, 1]` range by using a Rescaling layer.

In [ ]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

Note: The Keras Preprocessing utilities and layers introduced in this section are currently experimental and may change.

There are two ways to use this layer. You can apply it to the dataset by calling map:

Or, you can include the layer inside your model definition, which can simplify deployment. Let's use the second approach here.

Note: you previously resized images using the `image_size` argument of `image_dataset_from_directory`. If you want to include the resizing logic in your model as well, you can use the [Resizing](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Resizing) layer.

## Overfitting

In the plots above, the training accuracy is increasing linearly over time, whereas validation accuracy stalls around 60% in the training process. Also, the difference in accuracy between training and validation accuracy is noticeable—a sign of [overfitting](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit).

When there are a small number of training examples, the model sometimes learns from noises or unwanted details from training examples—to an extent that it negatively impacts the performance of the model on new examples. This phenomenon is known as overfitting. It means that the model will have a difficult time generalizing on a new dataset.

There are multiple ways to fight overfitting in the training process. In this tutorial, you'll use *data augmentation* and add *Dropout* to your model.

## Data augmentation

Overfitting generally occurs when there are a small number of training examples. [Data augmentation](https://www.tensorflow.org/tutorials/images/data_augmentation) takes the approach of generating additional training data from your existing examples by augmenting them using random transformations that yield believable-looking images. This helps expose the model to more aspects of the data and generalize better.

You will implement data augmentation using the layers from `tf.keras.layers.experimental.preprocessing`. These can be included inside your model like other layers, and run on the GPU.

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

You will use data augmentation to train a model in a moment.

## Dropout

Another technique to reduce overfitting is to introduce [Dropout](https://developers.google.com/machine-learning/glossary#dropout_regularization) to the network, a form of *regularization*.

When you apply Dropout to a layer it randomly drops out (by setting the activation to zero) a number of output units from the layer during the training process. Dropout takes a fractional number as its input value, in the form such as 0.1, 0.2, 0.4, etc. This means dropping out 10%, 20% or 40% of the output units randomly from the applied layer.

Let's create a new neural network using `layers.Dropout`, then train it using augmented images.

In [ ]:
num_classes = len(class_names) # 2: varroa or healthy

model = Sequential([
  data_augmentation,
  layers.experimental.preprocessing.Resizing(img_height, img_width, interpolation='bilinear'),
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="dense_varroa")
])

## Compile and train the model

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs = 200
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
model.summary()

## Visualize training results

After applying data augmentation and Dropout, there is less overfitting than before, and training and validation accuracy are closer aligned. 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#from google.colab import files
model_fname="flower"
model.save(model_fname)
#!zip -r /tmp/flower.zip /tmp/flower
#files.download('/tmp/flower.zip')



```
# This is formatted as code
```

# Intel OpenVINO toolkit Installation and Validations

In [ ]:
print("This will take a while...")
!wget -O openvino_key https://apt.repos.intel.com/openvino/2021/GPG-PUB-KEY-INTEL-OPENVINO-2021?elq_cid=6770273_ts1607381885691&erpm_id=9830841_ts1607381885691&elq_cid=6770273_ts1607381960247&erpm_id=9830841_ts1607381960247 
!apt-key add openvino_key
#!apt-key -q list
!echo "deb https://apt.repos.intel.com/openvino/2021 all main" | tee /etc/apt/sources.list.d/intel-openvino-2021.list
!apt update -q
!apt-cache search intel-openvino-dev-ubuntu18
!apt-get install intel-openvino-dev-ubuntu18-2021.2.200 -y -q
!pip install -U --no-deps --quiet openvino

#unknown bug - this is just a patch
!cp /opt/intel/openvino_2021/deployment_tools/inference_engine/external/tbb/lib/libtbb.so /usr/lib/x86_64-linux-gnu/libtbb.so
!cp /opt/intel/openvino_2021/deployment_tools/inference_engine/external/tbb/lib/libtbb.so.2 /usr/lib/x86_64-linux-gnu/libtbb.so.2
!ldconfig
print("Installation Completed...")
#%env we can set environmental variable with this

#Run the validation
!ls /opt/intel/
#Run the Validation Demo code.
demo_cmd = "/opt/intel/openvino_2021/deployment_tools/demo/demo_squeezenet_download_convert_run.sh"
import os
import subprocess
import shutil
output = subprocess.check_output(demo_cmd, shell=True)
print (output.decode('utf-8'))     

## Convert to OpenVINO IR format

In [ ]:
source_cmd= "/opt/intel/openvino_2021/bin/setupvars.sh"

# Setup model optimizer command ...
ir_name = "flower"
ir_data_type = "FP16" #32
ir_out_dir = f"/content/IR_models/{ir_data_type}"
ir_input_shape = "[1,50,50,3]"
mo_cmd = f"/opt/intel/openvino_2021/deployment_tools/model_optimizer/mo_tf.py \
      --saved_model_dir {model_fname} \
      --input_shape {ir_input_shape} \
      --data_type {ir_data_type} \
      --output_dir {ir_out_dir}  \
      --model_name {ir_name}"
print ("Running model optimizer to convert model to OpenVINO IR format ....")
print("\n--".join(mo_cmd.split("--")))

output = subprocess.check_output(source_cmd+" && "+mo_cmd, shell=True)
print (output.decode('utf-8'))     

## Convert OpenVINO IR to Blob format 

In [ ]:
!cp /opt/intel/openvino_2021.2.200/deployment_tools/inference_engine/lib/intel64/libmyriadPlugin.so /usr/lib/x86_64-linux-gnu/libmyriadPlugin.so
!source /opt/intel/openvino_2021/bin/setupvars.sh && /opt/intel/openvino_2021/deployment_tools/inference_engine/lib/intel64/myriad_compile -m {ir_out_dir}/flower.xml -ip U8 -VPU_NUMBER_OF_SHAVES 4 -VPU_NUMBER_OF_CMX_SLICES 4

In [ ]:
!ls /content/IR_models/FP16/
#download the blob file to local directory.
from google.colab import files
files.download(f"{ir_out_dir}/flower.blob")

#if you want everything - uncomment these two lines
#!zip -r flower.zip flower 
#files.download('flower.zip')


```
//#iamintel
//References: 
//COLAB for OpenVINO 
//https://colab.research.google.com/drive/1AwkhrLkjnmPDwTOGBlPSPs5FhycEr__d?authuser=1#scrollTo=V2iIEiRfjGBE
//Original Tensorflow Classification Tutorial:
//https://www.tensorflow.org/tutorials/images/classification
//First release: Jan 27, 2021
//Last update: March 30, 2021

#include <iostream>
using namespace std;

int main() {
    string author="Raymond Lo, PhD";
    string role="Intel OpenVINO Edge AI Evangelist";
    string linkedin="https://www.linkedin.com/in/raymondlo84/";

    cout << "Author:" << author << endl;
    cout << "Role:" << role << endl;
    cout << "LinkedIn: " << linkedin << endl;
    return 0; 
}

/**
  Except as otherwise noted, the content of this page is licensed 
  under the Creative Commons Attribution 4.0 License, and code
  samples are licensed under the Apache 2.0 License. 
**/

```

